In [1]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

In [ ]:
# 1. 대상 URL과 헤더 설정
url = "https://www.moviechart.co.kr/rank/realtime/index/image"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# 2. 페이지 요청
movie_ranking = requests.get(url, headers=headers)

if movie_ranking.status_code == 200:
    print('✅ 페이지를 성공적으로 불러왔습니다!')
    
    # .content를 사용하여 바이트 데이터를 BeautifulSoup에 전달 (인코딩에 유리)
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')

    # 3. 영화 리스트가 담긴 요소 찾기 
    #content > div.wArea.space > div.movieBox > ul > li:nth-child(1)
    # (선택자에서 반복되는 부분인 'li'를 타겟팅합니다)
    
    movies = soup.select(".movieBox-list .movieBox-item")

    print(f"현재 상영 중인 영화 {len(movies)}개를 발견했습니다.\n")

    for i, movie in enumerate(movies, 1):
        # 4. 제목 추출 
        # li 태그 내부에서 제목이 있는 클래스나 태그를 찾습니다.
        title_tag = movie.select_one(".title a")
        title = title_tag.get_text(strip=True) if title_tag else "제목 정보 없음"

        # 5. 이미지 URL 추출
        # 작성하신 선택자의 마지막 부분인 'img'를 찾습니다.
        img_tag = movie.select_one("a > img")
        img_url = img_tag['src'] if img_tag else "이미지 없음"

        # 상대 경로 처리 (주소가 /로 시작하면 도메인 붙여주기)
        if img_url.startswith('/'):
            img_url = "https://www.moviechart.co.kr" + img_url

        print(f"{i}위: {title}")
        print(f"🔗 포스터 주소: {img_url}")
        print("-" * 50)

else:
    print(f"❌ 페이지 호출 실패! (에러 코드: {movie_ranking.status_code})")

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  # 판다스 불러오기

url = "https://www.moviechart.co.kr/rank/realtime/index/image"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

movie_ranking = requests.get(url, headers=headers)

if movie_ranking.status_code == 200:
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')
    movies = soup.select(".movieBox-list .movieBox-item")
    
    # 1. 데이터를 담을 빈 리스트 준비
    movie_results = []

    for i, movie in enumerate(movies, 1):
        title_tag = movie.select_one(".title a")
        title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

        img_tag = movie.select_one("a > img")
        img_url = img_tag['src'] if img_tag else "이미지 없음"
        if img_url.startswith('/'):
            img_url = "https://www.moviechart.co.kr" + img_url

        # 2. 딕셔너리 형태로 리스트에 추가
        movie_results.append({
            '순위': i,
            '영화제목': title,
            '포스터링크': img_url
        })

    # 3. 데이터프레임 생성
    df = pd.DataFrame(movie_results)

    # 결과 출력
    print(df)

    # # 4. 엑셀(CSV) 파일로 저장 (인코딩 설정 주의!)
    # df.to_csv("movie_rank.csv", index=False, encoding="utf-8-sig")
    # print("\n✅ movie_rank.csv 파일로 저장되었습니다!")

else:
    print("페이지 호출 실패")

    순위   영화제목                                              포스터링크
0    1  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
1    2  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
2    3  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
3    4  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
4    5  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
5    6  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
6    7  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
7    8  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
8    9  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
9   10  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
10  11  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
11  12  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
12  13  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
13  14  제목 없음  https://www.moviechart.co.kr/thumb?width=178&h...
14  15  제목 없음  https://ww